# Submit Prediction to Kaggle

In [8]:
from pandas import DataFrame, read_csv, to_numeric
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from bigml.api import BigML
import kaggle
from time import sleep

%matplotlib inline

In [ ]:
compression = True  #Do we compress the csv files?

version='v1.3'

big_ml_project = 'project/5d94a428eba31d460c00023f'

In [ ]:
if compression:
    compress='.bz2'
else:
    compress=''

filenames = ["",""]
  
filename_fulltrain = 'gmsc-fulltrain-' + version + '.csv'+ compress
filename_train = 'gmsc-train80-' + version + '.csv'+ compress
filename_valid = 'gmsc-valid20-' + version + '.csv'+ compress
filename_test = 'gmsc-test-' + version + '.csv'+ compress
filename_valid_prediction = 'gmsc-valid-pred-' + version + '.csv'+ compress
filename_test_prediction = 'gmsc-test-pred-' + version + '.csv'+ compress

filename_bigml_ids = 'bigml-' + version + '.csv'+ compress

## Retrieve model and test dataset ids from Big ML

In [ ]:
api = BigML(project=big_ml_project)

bigml_ids = read_csv(filename_bigml_ids, squeeze=True, index_col=0)

model = api.get_ensemble(bigml_ids['model'])
test_ds = api.get_dataset(bigml_ids['test dataset'])

## Create a batch prediction on the test dataset

In [ ]:
batch_prediction = api.create_batch_prediction(model, test_ds, {
    "name": "my batch prediction" + version,
    "all_fields": False,
    "header": True,
    "confidence": True,
    "probabilities": True}                      )
api.ok(batch_prediction)
api.download_batch_prediction(batch_prediction,
                                filename=filename_test_prediction)
print('batch prediction:Done')

## Open prediction file

In [ ]:
prediction=read_csv(filename_test_prediction,index_col=0)

## Send prediction to Kaggle

In [ ]:
kaggle_prediction=DataFrame(index=prediction.index)
kaggle_prediction['Probability']=prediction['1 probability']

kaggle_prediction_file="kaggleprediction.csv"
kaggle_prediction.to_csv(kaggle_prediction_file,index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "Ensemble-" + str(version), "GiveMeSomeCredit")

## Retrieve last score

In [9]:
for i in range(5):
    error = kaggle.api.competitions_submissions_list("GiveMeSomeCredit")[0]['errorDescription']
    status = kaggle.api.competitions_submissions_list("GiveMeSomeCredit")[0]['status']
    print('status:', status, ', error:', error)
    sleep(1)


score = kaggle.api.competitions_submissions_list("GiveMeSomeCredit")[0]['publicScore']

status complete error: None
status complete error: None
status complete error: None
status complete error: None
status complete error: None
status complete error: None
status complete error: None
status complete error: None
status complete error: None
status complete error: None


In [6]:
kaggle.api.competitions_submissions_list("GiveMeSomeCredit")[0]

{'ref': 12980578,
 'totalBytes': 1749831,
 'date': '2019-10-24T13:43:47.297Z',
 'description': 'Ensemble-v1.3',
 'errorDescription': None,
 'fileName': 'kaggleprediction.csv',
 'publicScore': '0.85512',
 'privateScore': '0.86063',
 'status': 'complete',
 'submittedBy': 'Guillaume Meurisse',
 'submittedByRef': 'guillaume33',
 'teamName': 'Guillaume Meurisse',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/12980578/12980578.raw'}